In [52]:
import cv2
import os
import numpy as np


from skimage.feature import hog
from skimage import data, exposure
import matplotlib.image as mpimg

dirpath = "C:/Users/Carlos/Documents/esan/2018-1/Inteligencia Artificial Avanzada/Trabajo Final/iaa/Dataset_Arandanos/Hojas enfermas/"
Vectores_caract_LBP = [] #nuevo
carpeta = (os.listdir(dirpath))
n = 3

ext = ".jpg"

for file in carpeta: 
    image = mpimg.imread(dirpath + file) # imagenes en color
    image_EG = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY) #Se pasa a blanco y negro
    image_EG2 = cv2.medianBlur(image_EG, n) # Se realizar un  filtro de ruido usando la media para que todas las imagenes entren sin ruido
    image_EG3 = cv2.addWeighted(image_EG, 1.5, image_EG2, -0.5, 0, image_EG) # realzamos detalles
    image_EG4 = cv2.resize(image_EG3, (200, 200))# se aplica un rescalado a todas la imagenes para que entren con un tam
    nombre = file[:file.find(ext)]
    lbp = ft.local_binary_pattern(image_EG4, lbp_sampling_points, lbp_sampling_radius, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, lbp_sampling_points + 3), range=(0, lbp_sampling_points + 2))
    # normalizar el histograma
    hist = hist.astype("float")
    eps=1e-7
    hist /= (hist.sum() + eps)
    Vectores_caract_LBP.append(hist)

len(Vectores_caract_LBP)

344

In [53]:
import cv2
import os
import numpy as np


from skimage.feature import hog
from skimage import data, exposure
import matplotlib.image as mpimg

dirpath = "C:/Users/Carlos/Documents/esan/2018-1/Inteligencia Artificial Avanzada/Trabajo Final/iaa/Dataset_Arandanos/Hojas sanas/"
Vectores_caract_LBP2 = [] #nuevo
carpeta = (os.listdir(dirpath))
n = 3

ext = ".jpg"

for file in carpeta: 
    image = mpimg.imread(dirpath + file) # imagenes en color
    image_EG = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY) #Se pasa a blanco y negro
    image_EG2 = cv2.medianBlur(image_EG, n) # Se realizar un  filtro de ruido usando la media para que todas las imagenes entren sin ruido
    image_EG3 = cv2.addWeighted(image_EG, 1.5, image_EG2, -0.5, 0, image_EG) # realzamos detalles
    image_EG4 = cv2.resize(image_EG3, (200, 200))# se aplica un rescalado a todas la imagenes para que entren con un tam
    nombre = file[:file.find(ext)]
    lbp = ft.local_binary_pattern(image_EG4, lbp_sampling_points, lbp_sampling_radius, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, lbp_sampling_points + 3), range=(0, lbp_sampling_points + 2))
    # normalizar el histograma
    hist = hist.astype("float")
    eps=1e-7
    hist /= (hist.sum() + eps)
    Vectores_caract_LBP2.append(hist)

len(Vectores_caract_LBP2)

261

In [54]:
Vectores_caract_LBP2.extend(Vectores_caract_LBP)
X = np.array(Vectores_caract_LBP2)

valores = np.array([[1],[0]])
Y = np.repeat(valores, [344,261])

In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.20, random_state=42)

In [56]:
# Import `StandardScaler` from `sklearn.preprocessing`
from sklearn.preprocessing import StandardScaler

# Define the scaler 
scaler = StandardScaler().fit(X_train)

# Scale the train set
X_train = scaler.transform(X_train)

# Scale the test set
X_test = scaler.transform(X_test)

SVM


In [57]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
clf = clf.fit(X_train, y_train)

In [58]:
y_pred=clf.predict(X_test)

In [59]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7024793388429752

REDES NEURONALES


In [60]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15,), random_state=1)

clf.fit(X_train, y_train) 

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(15,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [61]:
y_pred=clf.predict(X_test)

In [62]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6776859504132231

RANDOM FOREST

In [63]:
from sklearn.ensemble import RandomForestRegressor
# Crear una instancia del modelo con 1000 árboles de decisión
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Entrenar al modelo en los datos de entrenamiento
rf.fit(X_train, y_train);

In [64]:
# Usa el método de predicción del bosque en los datos de pruebaa
predictions = rf.predict(X_test)
# Calcula los errores absolutos
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
print('Accuracy: ', 1- round(np.mean(errors), 2))

Mean Absolute Error: 0.34 degrees.
Accuracy:  0.6599999999999999
